In [9]:
import subprocess
subprocess.check_output(["/home/pi/bin/aws", "--version"], stderr=subprocess.STDOUT)

b'aws-cli/1.11.65 Python/2.7.9 Linux/4.4.32v7-aufs botocore/1.5.28\n'

In [52]:
import requests, argparse, json, logging, sys, os, sha256_tree_hash
import subprocess

BACKUP_TEMP_FOLDER='/media/usb2/backup'
PART_SIZE=67108864
AWS_VAULT='raspi'

ARCHIVE_PREFIX='/tmp/archive.gz.part_'
_args = {}
logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s')
_logger = logging.getLogger(__name__)
_logger.setLevel(logging.INFO)

def _start_request():
    out = subprocess.check_output(["/home/pi/bin/aws", 
                        "glacier", 
                        "initiate-multipart-upload", 
                        "--vault-name=raspi", 
                        "--account-id=-", 
                        "--archive-description=test", 
                        "--part-size=%s" % PART_SIZE ])
    
    return json.load(out)['uploadId']

def _complete_request(upload_id, checksum):
    stats = os.stat(_args.file)
    archive_size = stats.st_size
    out = subprocess.check_output(["/home/pi/bin/aws", 
                        "glacier", 
                        "complete-multipart-upload", 
                        "--vault-name=raspi", 
                        "--account-id=-",
                        "--checksum=%s" % checksum,
                        "--upload-id=%s" % upload_id,
                        "--archive-size=%s" % archive_size ])
    return out

def multi_upload(filename, upload_id):
    stats = os.stat(_args.file)
    archive_size = stats.st_size
    parts = archive_size // PART_SIZE
    if archive_size % PART_SIZE > 0:
        parts += 1
        
    sha256_parts = []
    dd = subprocess.Popen(['dd', 
          'if=%s' % filename, 
          'bs=2M']
         , stdout=subprocess.PIPE )
    
    out = subprocess.check_output(['split', 
          '--suffix-length=1', 
          '--numeric-suffixes=0', 
          '--bytes=%s' % PART_SIZE, 
          '-', 
          ARCHIVE_PREFIX]
         , stdin = dd.stdout )
    
    start=0
    end=0
    for i in range(parts):
        part_filename = ARCHIVE_PREFIX + str(i)
        start = start * PART_SIZE
        stats = os.stat(part_filename)
        end = start + stats.st_size - 1
        _logger.info(start)
        _logger.info(end)
        
        chunks = sha256_tree_hash.get_chunks_sha256_hashes(part_filename)
        checksum = sha256_tree_hash.compute_sha256_tree_hash(chunks)
        sha256_parts.append(checksum)
        
        call(['/home/pi/bin/aws',
              'glacier', 
              'upload-multipart-part',
              '--vault-name=raspi',
              '--account-id=-',
              '--body', part_filename,
              '--upload-id', upload_id,
              '--checksum', checksum,
              '--range', 'bytes {}-{}/*'.format(start, end)])
        start += 1

    complete_checksum = sha256_tree_hash.compute_sha256_tree_hash( sha256_parts )
    return complete_checksum
    
def _main():
    pass

if __name__=='__main__':   
    _parser = argparse.ArgumentParser()
    _parser.add_argument('-f', action='store', dest='file', type=str, help='File to upload')
    _parser.add_argument('-m', action='store', dest='descr', type=str, help='Description')
    _parser.add_argument('-d', '--debug', action='store_true', dest='debug', help='More logging on console')
    _args = _parser.parse_args()
    
    if _args.debug:
        _logger.setLevel(logging.DEBUG)
    
    _main()

In [53]:
multi_upload('/home/pi/script/test.tmp', 123)

TypeError: Can't convert 'int' object to str implicitly

In [36]:
import os
os.listdir('/tmp')

['.X11-unix',
 'archive.gz.part_01',
 '.Test-unix',
 '.ICE-unix',
 '.font-unix',
 'pulse-PKdhtXMmr18n',
 '.XIM-unix',
 'jna-3577',
 'systemstats.pid',
 'hsperfdata_pi',
 'archive.gz.part_00',
 'owncloud-server-oc1zmxlq1b4f-cron.lock',
 'marketbest',
 'tmux-1000',
 'awscli.response']

In [51]:
'{}'.format(1)

'1'

In [67]:
import sha256_tree_hash, codecs

C = sha256_tree_hash.get_chunks_sha256_hashes('/home/pi/script/test.tmp')
C[0]

b'0\xe1IU\xeb\xf15"f\xdc/\xf8\x06~h\x10F\x07\xe7P\xab\xb9\xd3\xb3e\x82\xb8\xaf\x90\x9f\xcbX'

In [66]:
codecs.getdecoder('hex')('30e14955ebf1352266dc2ff8067e68104607e750abb9d3b36582b8af909fcb58')[0]

b'0\xe1IU\xeb\xf15"f\xdc/\xf8\x06~h\x10F\x07\xe7P\xab\xb9\xd3\xb3e\x82\xb8\xaf\x90\x9f\xcbX'

In [1]:
import subprocess
gzip = subprocess.Popen(["gzip", "-1c", '/tmp/test100M.tmp'], stdout=subprocess.PIPE)
out = subprocess.Popen(['dd', 'of=/tmp/test100M.gz', 'bs=2M'], stdin=gzip.stdout, stdout=subprocess.PIPE)
out.wait()
gzip.stdout.close()
print(out.communicate()[0])

b''
